In [ ]:
# BLOCK 1 : FIND THE PATH OF THE TROPICAL CYCLONE DATA THAT WAS DOWNLOADER OVER WIS2
import os
import glob

def find_latest_file_with_path(root_dir, sub_path, extension):
    # Construct the search pattern to include the specific path and extension
    search_pattern = os.path.join(root_dir, '**', sub_path, f'*.{extension}')
    # Use glob to find all matching files recursively
    matching_files = glob.glob(search_pattern, recursive=True)
    if not matching_files:
        return None
    # Find the latest file based on the modification time
    latest_file = max(matching_files, key=os.path.getmtime) 
    return latest_file

# Set the root directory and the specific sub-path
root_dir = '/root/downloads'
sub_path = 'forecast/medium-range/deterministic/trajectory'
extension = 'bin'

# Call the function
latest_file = find_latest_file_with_path(root_dir, sub_path, extension)

if latest_file:
    print(f"The latest .bin file with the path '{sub_path}' is: {latest_file}")
else:
    print(f"No .bin files found with the path '{sub_path}'.")

In [ ]:
# BLOCK 2 : PARSE THE DATA 
infile = latest_file

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import LAND, OCEAN, COASTLINE
from datetime import datetime as dt, timedelta
from eccodes import codes_bufr_new_from_file, codes_set, codes_get, codes_get_array, codes_get_double, codes_get_long
from eccodes import CODES_MISSING_DOUBLE
from functools import partial
import json
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import colormaps
import numpy as np
import pyproj
from shapely.geometry import shape, Point, MultiPoint
from shapely.ops import transform

# set map bounds, this will be updated as part of reading the data
bounds = [180,-180, 90, -90]


# open and create bufr handle
fh = open(infile,"rb")
bufr = codes_bufr_new_from_file(fh)
# unpack
codes_set(bufr,"unpack",True)
compressed = codes_get(bufr, 'compressedData')